In [1]:
import open3d as o3d
# o3d.visualization.webrtc_server.enable_webrtc()
from open3d.web_visualizer import draw
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [ ]:
def quat_from_two_vectors_wxyz(a, b):
    # quaternion rotate one vec to another
    a /= np.linalg.norm(a)
    b /= np.linalg.norm(b)

    v = np.cross(a, b)
    w = 1.0 + np.dot(a, b)

    q = np.array([w, v[0], v[1], v[2]], dtype=float)
    q /= np.linalg.norm(q)
    return q 

In [27]:
import numpy as np
import open3d as o3d

pcd = o3d.io.read_point_cloud("/home/roman/ba/huk_scenes/audi_nbg/sparse/0/points3D.ply")
# pcd = o3d.io.read_point_cloud("/home/roman/ba/datasets/ref_real/sedan/sparse/0/points3D.ply")

pcd_filtered = pcd.voxel_down_sample(voxel_size=0.10)
pcd_filtered, _ = pcd_filtered.remove_statistical_outlier(nb_neighbors=30, std_ratio=2.0)

plane_model, inliers = pcd.segment_plane(
    distance_threshold=0.08,
    ransac_n=3,
    num_iterations=2000
)

[a, b, c, d] = plane_model
print("Plane:", plane_model, "inliers:", len(inliers))

n = np.array([a,b,c], float)

z_axis = np.array([0.0, 0.0, 1.0])

q_wxyz = quat_from_two_vectors_wxyz(n, z_axis)
R = o3d.geometry.get_rotation_matrix_from_quaternion(q_wxyz)

pcd.rotate(R, center=pcd.get_center())
o3d.io.write_point_cloud("audi_nbg_rotated.ply", pcd)
draw(pcd)

Plane: [-0.03306963  0.93876352  0.34297152 -1.69238756] inliers: 92226
[Open3D INFO] Window window_11 created.


WebVisualizer(window_uid='window_11')

In [31]:

from dataclasses import dataclass
from typing import List, Tuple
import struct

@dataclass
class Point3D:
    id: int
    xyz: Tuple[float, float, float]  # Assuming xyz is a tuple of three floats
    rgb: Tuple[int, int, int]  # Assuming rgb is a tuple of three integers representing color
    error: float
    image_ids: List[int]  # Assuming image_ids is a list of integers
    point2D_idxs: List[int]  # Assuming point2D_idxs is a list of integers

def read_next_bytes(fid, num_bytes, format_char_sequence, endian_character="<"):
    """Read and unpack the next bytes from a binary file.
    :param fid:
    :param num_bytes: Sum of combination of {2, 4, 8}, e.g. 2, 6, 16, 30, etc.
    :param format_char_sequence: List of {c, e, f, d, h, H, i, I, l, L, q, Q}.
    :param endian_character: Any of {@, =, <, >, !}
    :return: Tuple of read and unpacked values.
    """
    data = fid.read(num_bytes)
    return struct.unpack(endian_character + format_char_sequence, data)

In [32]:
def read_points3D_binary(path_to_model_file):
    """
    see: src/base/reconstruction.cc
        void Reconstruction::ReadPoints3DBinary(const std::string& path)
        void Reconstruction::WritePoints3DBinary(const std::string& path)
    """
    points3D = {}
    with open(path_to_model_file, "rb") as fid:
        num_points = read_next_bytes(fid, 8, "Q")[0]
        for point_line_index in range(num_points):
            binary_point_line_properties = read_next_bytes(
                fid, num_bytes=43, format_char_sequence="QdddBBBd")
            point3D_id = binary_point_line_properties[0]
            xyz = np.array(binary_point_line_properties[1:4])
            rgb = np.array(binary_point_line_properties[4:7])
            error = np.array(binary_point_line_properties[7])
            track_length = read_next_bytes(
                fid, num_bytes=8, format_char_sequence="Q")[0]
            track_elems = read_next_bytes(
                fid, num_bytes=8 * track_length,
                format_char_sequence="ii" * track_length)
            image_ids = np.array(tuple(map(int, track_elems[0::2])))
            point2D_idxs = np.array(tuple(map(int, track_elems[1::2])))
            points3D[point3D_id] = Point3D(
                id=point3D_id, xyz=xyz, rgb=rgb,
                error=error, image_ids=image_ids,
                point2D_idxs=point2D_idxs)
    return points3D

In [36]:
def read_points3D_bin(path: str):
    """
    COLMAP points3D.bin format (little-endian):
      Q            : num_points
      repeat num_points:
        Q          : point3D_id
        d d d      : xyz
        B B B      : rgb
        d          : error
        Q          : track_length
        repeat track_length:
          I I      : image_id, point2D_idx   (ignored here)
    """
    with open(path, "rb") as f:
        num_points = struct.unpack("<Q", f.read(8))[0]

        xyz = np.empty((num_points, 3), dtype=np.float64)
        rgb = np.empty((num_points, 3), dtype=np.uint8)

        for i in range(num_points):
            _pid = struct.unpack("<Q", f.read(8))[0]
            x, y, z = struct.unpack("<ddd", f.read(24))
            r, g, b = struct.unpack("<BBB", f.read(3))
            _err = struct.unpack("<d", f.read(8))[0]

            track_len = struct.unpack("<Q", f.read(8))[0]
            f.seek(track_len * 8, 1)  # each track elem = 2*uint32 = 8 bytes

            xyz[i] = (x, y, z)
            rgb[i] = (r, g, b)

    return xyz, rgb

def write_ply_ascii(path: str, xyz: np.ndarray, rgb: np.ndarray):
    n = xyz.shape[0]
    with open(path, "w", encoding="utf-8") as f:
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write(f"element vertex {n}\n")
        f.write("property float x\nproperty float y\nproperty float z\n")
        f.write("property uchar red\nproperty uchar green\nproperty uchar blue\n")
        f.write("end_header\n")
        for (x, y, z), (r, g, b) in zip(xyz, rgb):
            f.write(f"{x:.6f} {y:.6f} {z:.6f} {int(r)} {int(g)} {int(b)}\n")

In [37]:
xyz, rgb = read_points3D_bin("/home/roman/ba/huk_scenes_stablenormal/audi_silver/sparse/0/points3D.bin")
write_ply_ascii("/home/roman/ba/huk_scenes_stablenormal/audi_silver/sparse/0/points3D.ply", xyz, rgb)

[5287:440][3461] (dtls_transport.cc:529): DtlsTransport[0|1|__]: OnWritableState() called in state DTLS_TRANSPORT_CLOSED.
[5288:727][3461] (dtls_transport.cc:529): DtlsTransport[0|1|__]: OnWritableState() called in state DTLS_TRANSPORT_CLOSED.
[5288:759][3461] (dtls_transport.cc:529): DtlsTransport[0|1|__]: OnWritableState() called in state DTLS_TRANSPORT_CLOSED.
[5288:775][3461] (dtls_transport.cc:529): DtlsTransport[0|1|__]: OnWritableState() called in state DTLS_TRANSPORT_CLOSED.
[5288:799][3461] (dtls_transport.cc:529): DtlsTransport[0|1|__]: OnWritableState() called in state DTLS_TRANSPORT_CLOSED.
[5288:804][3461] (dtls_transport.cc:529): DtlsTransport[0|1|__]: OnWritableState() called in state DTLS_TRANSPORT_CLOSED.
[5288:808][3461] (dtls_transport.cc:529): DtlsTransport[0|1|__]: OnWritableState() called in state DTLS_TRANSPORT_CLOSED.
[5288:814][3461] (dtls_transport.cc:529): DtlsTransport[0|1|__]: OnWritableState() called in state DTLS_TRANSPORT_CLOSED.
[5288:922][3461] (dtls_t

In [38]:
points = read_points3D_binary("/home/roman/ba/huk_scenes_stablenormal/audi_silver/sparse/0/points3D.bin")

In [44]:
vals = list(points.values())

In [45]:
vals

[Point3D(id=4, xyz=array([ 4.31974276, -1.85006096,  1.42762857]), rgb=array([55, 65, 32]), error=array(2.73133126), image_ids=array([149, 150,   1]), point2D_idxs=array([ 63, 129,   3])),
 Point3D(id=7, xyz=array([ 1.45469431, -2.31287665,  3.02735   ]), rgb=array([ 88, 102,  49]), error=array(1.91518441), image_ids=array([  2,   1, 149, 150, 151]), point2D_idxs=array([    6,     6,  3704,  4033, 17273])),
 Point3D(id=8, xyz=array([ 1.57925506, -2.30973327,  2.99266066]), rgb=array([76, 88, 44]), error=array(0.91515727), image_ids=array([  2,   1, 153]), point2D_idxs=array([    7,     7, 15447])),
 Point3D(id=9, xyz=array([ 1.59957438, -2.17880266,  2.73618758]), rgb=array([55, 68, 20]), error=array(2.08015372), image_ids=array([2, 1]), point2D_idxs=array([8, 8])),
 Point3D(id=10, xyz=array([ 1.6742141 , -2.1187703 ,  2.60749175]), rgb=array([69, 76, 36]), error=array(2.33544069), image_ids=array([2, 1]), point2D_idxs=array([9, 9])),
 Point3D(id=11, xyz=array([ 1.86354553, -2.19119466

In [53]:
points = read_points3D_binary("/home/roman/ba/huk_scenes_stablenormal/audi_silver/sparse/0/points3D.bin")

In [54]:
write_ply_ascii(points, "/home/roman/ba/huk_scenes_stablenormal/audi_silver/sparse/0/points3D.ply")

In [8]:
q_wxyz

array([ 0.84226826,  0.5390325 ,  0.0053049 , -0.        ])

In [ ]:
R = o3d.geometry.get_rotation_matrix_from_quaternion(q_wxyz)

pcd.rotate(R, center=pcd.get_center())

In [ ]:
# [-0.00211864  0.93661683  0.350349   -1.68627746]
# [-0.0089363   0.90801993  0.41883165 -1.77691084]

In [16]:
ref_env = o3d.io.read_point_cloud("/home/roman/ba/datasets/ref_real/sedan/envs/points3D.ply")

In [17]:
env_data = np.asarray(ref_env.points)

In [18]:
density = env_data.shape[0] / (env_data.max(axis=0) - env_data.min(axis=0)).prod()

In [19]:
density

np.float64(4.160198365056736)

In [ ]:
# env_bounds: [[-15.41093584, -9.52983853, -26.35259539], [28.42411913, 10.86091045, 17.71185112]]